![clothing_classification](clothing_classification.png)


Fashion Forward is a new AI-based e-commerce clothing retailer.
They want to use image classification to automatically categorize new product listings, making it easier for customers to find what they're looking for. It will also assist in inventory management by quickly sorting items.

As a data scientist tasked with implementing a garment classifier, your primary objective is to develop a machine learning model capable of accurately categorizing images of clothing items into distinct garment types such as shirts, trousers, shoes, etc.

In [8]:
# Run the cells below first

In [9]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics import Accuracy, Precision, Recall

In [10]:
# Load datasets
from torchvision import datasets
import torchvision.transforms as transforms

train_data = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_data = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

In [11]:
# Create DataLoaders for training and testing
batch_size = 64
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [12]:
# 1. Define the CNN architecture

class GarmentClassifier(nn.Module):
    def __init__(self):
        super(GarmentClassifier, self).__init__()
        # Convolutional layer: 1 input channel (grayscale), 32 output channels, kernel size 3
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
        # Rectified Linear Unit activation
        self.relu = nn.ReLU()
        # Pooling layer: 2x2 max pooling reduces spatial dimensions by half
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        # Fully connected layer:
        # Image dimensions reduce from 28x28 to 14x14 after pooling, so input features = 32 * 14 * 14
        # Output features = 10 (one per class)
        self.fc1 = nn.Linear(32 * 14 * 14, 10)
    
    def forward(self, x):
        # Apply convolution, activation, and pooling
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        # Flatten the output tensor for the fully connected layer
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return x

In [13]:
# 2. Training the CNN

# Use GPU if available, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GarmentClassifier().to(device)

# Define loss criterion and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop (2 epochs)
epochs = 2
model.train()
for epoch in range(epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()       # Clear previous gradients
        outputs = model(inputs)       # Forward pass
        loss = criterion(outputs, labels)   # Compute loss
        loss.backward()             # Backward pass
        optimizer.step()            # Update model parameters
        
        running_loss += loss.item()
    
    avg_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")

Epoch 1/2, Loss: 0.4354
Epoch 2/2, Loss: 0.3042


In [14]:
# 3. Testing the model 

model.eval()
predictions = []  
all_labels = []   

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        outputs = model(inputs)
        # Get the predicted class (highest score)
        _, preds = torch.max(outputs, 1)
        predictions.extend(preds.cpu().numpy().tolist())
        all_labels.extend(labels.cpu().numpy().tolist())

# Convert predictions and true labels to tensors for metric calculations
preds_tensor = torch.tensor(predictions)
labels_tensor = torch.tensor(all_labels)

# Define torchmetrics for accuracy, per-class precision, and per-class recall.
# We specify task="multiclass" and num_classes=10 (for FashionMNIST).
accuracy_metric = Accuracy(task="multiclass", num_classes=10)
precision_metric = Precision(task="multiclass", num_classes=10, average=None)
recall_metric = Recall(task="multiclass", num_classes=10, average=None)

# Calculate metrics
accuracy = accuracy_metric(preds_tensor, labels_tensor).item()
precision = precision_metric(preds_tensor, labels_tensor).tolist()  
recall = recall_metric(preds_tensor, labels_tensor).tolist()         

print(f"\nTest Accuracy: {accuracy:.4f}")
print("Per-class Precision:", precision)
print("Per-class Recall:", recall)


Test Accuracy: 0.8907
Per-class Precision: [0.8448275923728943, 0.9788944721221924, 0.879133403301239, 0.8385370373725891, 0.8109405040740967, 0.9856410026550293, 0.7114583253860474, 0.9333333373069763, 0.9624876379966736, 0.9628514051437378]
Per-class Recall: [0.8330000042915344, 0.9739999771118164, 0.7710000276565552, 0.9399999976158142, 0.8450000286102295, 0.9610000252723694, 0.6830000281333923, 0.9660000205039978, 0.9750000238418579, 0.9589999914169312]
